<a href="https://colab.research.google.com/github/IshtiSikder/Optimal-allocation-of-rescue-teams-for-hostage-situations/blob/test/toy_prob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
prob <- function(opSet = policySet, t, lam, eps, mu) {
  opTRM = matrix(0, nrow = t+1, ncol = t+1)
  for (i in 2:t) {
    opTRM[i, i-1] = sum(i*eps, opSet[i]*mu)
    opTRM[i, i+1] = lam      
  }
  opTRM[1, 2] = lam
  opTRM[t+1, t] = sum(t*eps, opSet[t+1]*mu)
  
  k = t+1
  steady_prob <- matrix(0, nrow=1, ncol=k)
  state <- matrix(0, nrow=1, ncol=k-1)
  for (i in 1:(k-1)) {
    if (i-2 < 0) {
      state[i] <- (lam/opTRM[i+1,i])
    } else {
      state[i] <- state[i-1]*(lam/opTRM[i+1,i])
    }
  }
  
  steady_prob[1] <- (1+sum(state))^-1
  for (i in 2:k) {
    steady_prob[i] <- state[i-1]*steady_prob[1]
  }
  return(steady_prob)
}

In [2]:
checkPolicy = function(policySet, policySetnew) {
  n = 0
  for (i in policySet == policySetnew) {
    if (i == 'FALSE') {
      n = 1
    } 
  } 
  if (n == 0) {
    return("TRUE")
  } else {
    return("FALSE")
  }
} 

In [3]:
t = 2
r = 2
lam = 0.2763
mu = 0.2829
eps = 0.1718

costT = 1
mult = 0.32

costR = mult*costT*(mu+eps)

In [4]:
aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  
for (k in 1:(r+1)) {
  for (i in 1:t) {
    aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)        
      }
    }
aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
aijMatrix[1,2,k] <- lam
}

for (k in 1:(r+1)) {
  for (i in 1:(t+1)) {
      
      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])      
    }
  }

In [5]:
print(aijMatrix)

, , 1

        [,1]    [,2]    [,3]
[1,] -0.2763  0.2763  0.0000
[2,]  0.1718 -0.4481  0.2763
[3,]  0.0000  0.3436 -0.3436

, , 2

        [,1]    [,2]    [,3]
[1,] -0.2763  0.2763  0.0000
[2,]  0.4547 -0.7310  0.2763
[3,]  0.0000  0.6265 -0.6265

, , 3

        [,1]    [,2]    [,3]
[1,] -0.2763  0.2763  0.0000
[2,]  0.4547 -0.7310  0.2763
[3,]  0.0000  0.9094 -0.9094



In [6]:
policyOld = array(3, dim = c(1,t+1))
policyNew = array(1, dim=c(1,t+1))

In [7]:
cat(policyOld,'\n\n')
cat(policyNew)

3 3 3 

1 1 1

In [8]:
policyOld = policyNew
policy = policyNew

A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]  
    }
newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])

newA

1,0.2763,-0.2763
1,-0.1718,0.4481
1,0.0000,-0.3436


In [9]:
q = array(0,dim=c(t+1,1))

for (i in 1:length(policy)) {

      q[i,1] = (policy[i]-1)*costR + (i-1)*eps*costT

}

q

0.0000
0.1718
0.3436


In [10]:
solution = solve(newA) %*% q

 newsolution = array(0, dim=c(t+1,1))
    
    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }
newsolution[t+1] = 0

gain = solution[1]

newsolution

-1.1309421
-0.4624174
0.0000000


In [11]:
gain

[1] 0.1847134

In [12]:
improveMatrix = array(0, dim=c(t+1,r+1))
cost = array(0,dim=c(t+1,r+1))

    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists

      cost[i,j] = (j-1)*costR + (i-1)*eps*costT
      
      improveMatrix[i,j] = cost[i,j] + aijMatrix[i,,j] %*% newsolution

      }
    }  

improveMatrix

0.1847134,0.3302174,0.4757214
0.1847134,0.1410917,0.2865957
0.1847134,0.1993995,0.2140856


In [13]:
for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }

policyNew

1,2,1


In [15]:
prob(opSet = policyNew, t=2, lam = 0.2763, eps = 0.1718, mu = 0.2829)

0.695497,0.2113105,0.0931925


In [20]:
# CTMC MDP model for rescue operational system!!!

prob <- function(opSet = policySet, t, lam, eps, mu) {
  opTRM = matrix(0, nrow = t+1, ncol = t+1)
  for (i in 2:t) {
    opTRM[i, i-1] = sum(i*eps, opSet[i]*mu)
    opTRM[i, i+1] = lam      
  }
  opTRM[1, 2] = lam
  opTRM[t+1, t] = sum(t*eps, opSet[t+1]*mu)
  
  k = t+1
  steady_prob <- matrix(0, nrow=1, ncol=k)
  state <- matrix(0, nrow=1, ncol=k-1)
  for (i in 1:(k-1)) {
    if (i-2 < 0) {
      state[i] <- (lam/opTRM[i+1,i])
    } else {
      state[i] <- state[i-1]*(lam/opTRM[i+1,i])
    }
  }
  
  steady_prob[1] <- (1+sum(state))^-1
  for (i in 2:k) {
    steady_prob[i] <- state[i-1]*steady_prob[1]
  }
  return(steady_prob)
}




# Check function to determine new policy is same with current or not 
checkPolicy = function(policySet, policySetnew) {
  n = 0
  for (i in policySet == policySetnew) {
    if (i == 'FALSE') {
      n = 1
    } 
  } 
  if (n == 0) {
    return("TRUE")
  } else {
    return("FALSE")
  }
} 


# Define MDP function (for basic policy)
MDP = function(lam, mu, eps, numRescue, numTerror, costT, RC, mult) {
  r = numRescue
  t = numTerror
  
  #calculating costR

  costR = mult*costT*(mu+eps)


  #########################################################
  #EDITS: 22ND FEB,2023
  #NEW SERVICE RATES STRUCTURE, ADD ORIGINAL ONE AS WELL
  aijMatrix = array(0, dim = c(t+1,t+1,r+1))
  for (k in 1:(r+1)) {
    for (i in 1:t) {
      aijMatrix[i,i+1,k] <- lam
      if (i > 1) {
        #if (k==4){
          #if(i==4){
            #cat('min(k-1,i-1)',min(k-1,i-1))}}
        aijMatrix[i,i-1,k] <- sum(min(k-1,i-1)*mu+(i-1)*eps)        
      }
    }
    aijMatrix[t+1,t,k] <- (min(k-1,t)*mu+t*eps)
    aijMatrix[1,2,k] <- lam
}

  for (k in 1:(r+1)) {
    for (i in 1:(t+1)) {
      
      aijMatrix[i,i,k] <- -sum(aijMatrix[i,,k])      
    }
  }
  #########################################################
  
  
  # Design the MDP algorithm
  # Value determination
  policy = array(0, dim = c(1,t+1)) 
  policyOld = array(0, dim = c(1,t+1))
  policyNew = array(2, dim=c(1,t+1)) # initially set all decision is 1 in first policy
  policyTrack = array(2, dim=c(1,t+1))
  iteration = 0
  gainMatrix = c()
  
  while (checkPolicy(policyOld, policyNew) == 'FALSE') {
    policyOld = policyNew
    policy = policyNew
    # Design the transition matrix regarding policy
    A = array(0, dim = c(t+1,t+1))
    for (i in 1:length(policy)) {
      A[i,] = aijMatrix[i,,(policy[i])]  
    }

    # Design the cost matrix regarding policy

    q = array(0,dim=c(t+1,1))
    for (i in 1:length(policy)) {

      if (RC == 'L'){
      cost = (policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost = sqrt(policy[i]-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost = ((policy[i]-1)^2)*costR + (i-1)*eps*costT
      }

      q[i,1] = cost
    }


    newA = cbind(array(1, dim=c(t+1,1)), -A[,1:t])
    solution = solve(newA) %*% q
    gain = solution[1] # First row of solution is gain value
    gainMatrix = append(gainMatrix, gain)
    
    # Policy improvement
    newsolution = array(0, dim=c(t+1,1))
    for (i in 1:t) {
      newsolution[i] = solution[i+1]
    }


    newsolution[t+1] = 0 # Set last v value as 0 to calcultate the test quantity in policy improvement
    improveMatrix = array(0, dim=c(t+1,r+1))
    cost = array(0,dim=c(t+1,r+1))

    for (i in 1:(t+1)) {
      for (j in 1:(r+1)) {
        # Cost when we have j rescue operation and i terrorists

        if (RC == 'L'){
      cost[i,j] = (j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'SR'){
      cost[i,j] = sqrt(j-1)*costR + (i-1)*eps*costT
      }

      if (RC == 'Q'){
      cost[i,j] = ((j-1)^2)*costR + (i-1)*eps*costT
      }

        improveMatrix[i,j] = cost[i,j] + aijMatrix[i,,j] %*% newsolution

      }
    }  
    
    # pick the minimum value among the decision
    policyNew = array(1, dim = c(1,t+1))
    for (i in 1:length(policyNew)) {
      policyNew[i] = which.min(improveMatrix[i,])
    }
    iteration = iteration + 1
    policyTrack = rbind(policyTrack, policyNew)
    policyNew
  }
  for (i in 1:length(policyNew)) {
    policyNew[i] = policyNew[i] - 1
  }
  for (i in 1:nrow(policyTrack)) {
    for (j in 1:ncol(policyTrack)) {
      policyTrack[i,j] = policyTrack[i,j] - 1      
    }

  steady = prob(opSet = policyNew, t=numTerror, lam = lam, eps = eps, mu = mu)

  gain = solution[1]

  }

  
   #return(list(policyNew,improveMatrix,cost,aijMatrix,newsolution))

   return (list(policyNew,gain,steady))
 
 }

In [21]:
MDP(lam=0.2763, mu=0.2829, eps=0.1718, numRescue=2, numTerror=2, costT=1, RC='L', mult=0.32)

0,1,0
0.5568979,0.245604,0.1974982


In [31]:
mult_test = function(cost_terror,rc,low_lim,up_lim,incr) {

prev_result = array(100,dim=c(1,11))

for (val in seq(low_lim,up_lim, by = incr)){

        output = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=2, numTerror=2, costT=cost_terror, RC=rc, mult=val)
     
        current_result = output[1]
        gain = output[2]
        steady = output[3]
         
        if (!(identical(current_result,prev_result))) {

          cat('\n\n','mult:',val,',','RC:',rc,',','\n\n')
          cat('optimal policy:\n')
          print(current_result)

          cat('Gain:\n')
          print(gain)

          cat('Steady state probs:\n')
          print(steady)

          prev_result = current_result

        }

    }

}

In [32]:
mult_test(1,'L',0,10,0.01)



 mult: 0 , RC: L , 

optimal policy:
[[1]]
     [,1] [,2] [,3]
[1,]    0    1    2

Gain:
[[1]]
[1] 0.09364119

Steady state probs:
[[1]]
          [,1]      [,2]       [,3]
[1,] 0.6349143 0.2800109 0.08507479



 mult: 0.32 , RC: L , 

optimal policy:
[[1]]
     [,1] [,2] [,3]
[1,]    0    1    0

Gain:
[[1]]
[1] 0.1720687

Steady state probs:
[[1]]
          [,1]     [,2]      [,3]
[1,] 0.5568979 0.245604 0.1974982



 mult: 0.42 , RC: L , 

optimal policy:
[[1]]
     [,1] [,2] [,3]
[1,]    0    0    0

Gain:
[[1]]
[1] 0.1847134

Steady state probs:
[[1]]
          [,1]      [,2]      [,3]
[1,] 0.4080363 0.3281154 0.2638483



ERROR: ignored

In [ ]:
output = MDP(lam = 0.2763, mu = 0.2829, eps = 0.1718, numRescue=2, numTerror=2, costT=cost_terror, RC=rc, mult=val)